In [0]:
from pyspark.sql.functions import count, sum, abs, round

# Análise de Volume por Tipo de Portabilidade
analise_volumetrica = (spark.table("cvm_p210.gold_cvm210_analytics")
    .filter("is_portabilidade = 'Sim'")
    .groupBy("classificacao_negocio")
    .agg(
        count("CNPJ_FUNDO").alias("quantidade_de_fundos"),
        round(sum(abs("fluxo_liquido")), 2).alias("valor_total_movimentado")
    )
    .orderBy("valor_total_movimentado", ascending=False)
)

print("📊 Resumo Executivo: Movimentações de Portabilidade (Janeiro/2026)")
display(analise_volumetrica)

📊 Resumo Executivo: Movimentações de Portabilidade (Janeiro/2026)


classificacao_negocio,quantidade_de_fundos,valor_total_movimentado
Portabilidade Receptora (Sucesso),671,5.081691097077E10
Portabilidade Defensiva (Perda),516,3.800668298762E10


In [0]:
# ==========================================================================================================
# NOTA ESTRATÉGICA - INTELIGÊNCIA DE PORTABILIDADE (CVM 210)
# ==========================================================================================================
# CONTEXTO: Embora a CVM não forneça a identificação direta das Instituições de Pagamento (IPs) de origem/destino
# devido ao sigilo bancário, este pipeline reconstrói a lógica de portabilidade através de fluxos líquidos anômalos.
#
# IMPORTÂNCIA DA SOLUÇÃO:
# 1. BLINDAGEM (RETENÇÃO): A identificação precoce de 'Portabilidade de Saída' é vital para a sobrevivência 
#    da gestora. Perder patrimônio líquido (PL) impacta diretamente na Taxa de Administração e na saúde do fundo.
#    O dado aqui gerado permite que o time de especialistas atue preventivamente para reter o investidor.
#
# 2. PROSPECÇÃO (ATAQUE): Identificar fluxos de entrada permite ao time de CRM e Especialistas agir no 
#    momento de maior confiança do investidor, consolidando o relacionamento e buscando 'Share of Wallet'.
#
# 3. INTEGRAÇÃO CRM: Este código não é apenas um relatório, é um motor de tarefas para o time comercial.


# O certo era ter a ag e conta do cliente para enviarmos o cliente que pediu portabilidade para seu especialista de investimento.
# ==========================================================================================================


from pyspark.sql.functions import col, abs

# 1. Filtragem de Alvos Críticos (Apenas Saída Massiva)
df_risco_saida = (spark.table("cvm_p210.gold_cvm210_analytics")
    .filter("classificacao_negocio = 'Portabilidade Defensiva (Perda)'")
    .orderBy(col("fluxo_liquido").asc()) # Do maior prejuízo para o menor
)

# 2. Simulação de Texto Profissional para o Especialista
def gerar_corpo_email_comercial(lista_fundos):
    texto = f"""
    ASSUNTO: [URGENTE] Monitoramento CVM 210 - Alerta de Evasão de Patrimônio
    
    Prezado Especialista de Investimentos,

    O monitoramento automatizado de portabilidades identificou fluxos de SAÍDA ATÍPICOS nas últimas competências. 
    Este comportamento sugere movimentações de portabilidade (CVM 210) que podem impactar severamente o PL gerido.

    DADOS DOS FUNDOS EM RISCO:
    {lista_fundos}

    AÇÃO RECOMENDADA:
    - Priorizar contato com os cotistas destes fundos via CRM.
    - Avaliar se a saída está atrelada à performance recente ou movimentação de mercado concorrente.
    - Ativar protocolo de retenção (Blindagem de Capital) para mitigar a continuidade dos resgates.

    A proatividade neste momento é a nossa principal ferramenta para garantir a estabilidade dos ativos sob gestão.

    Atenciosamente,
    Data Intelligence Bot - Databricks
    """
    return texto

# Coletando os 3 piores casos para o exemplo
top_riscos = df_risco_saida.limit(3).toPandas()
lista_str = ""
for index, row in top_riscos.iterrows():
    lista_str += f"- Fundo: {row['CNPJ_FUNDO']} | Perda Estimada: R$ {abs(row['fluxo_liquido']):,.2f}\n"

print(gerar_corpo_email_comercial(lista_str))